Importing requered libraries and Dataset

In [26]:
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import datasets, layers, models, losses
from sklearn.metrics import classification_report, f1_score
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Splitting the dataset into training and test dataset

In [27]:
(x_train,y_train),(x_test,y_test) = datasets.mnist.load_data()
x_train.shape

(60000, 28, 28)

Padding the dataset with zeroes and 8 bit values to make test and train dataset to recieve 32 by 32 images

In [28]:
x_train = tf.pad(x_train, [[0, 0], [2,2], [2,2]])/255
x_test = tf.pad(x_test, [[0, 0], [2,2], [2,2]])/255
x_train.shape

TensorShape([60000, 32, 32])

Expanding the tensor 

In [29]:
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
x_train.shape

TensorShape([60000, 32, 32, 1])

Introducing the validation set

In [30]:
x_val = x_train[-2000:,:,:,:] 
y_val = y_train[-2000:] 
x_train = x_train[:-2000,:,:,:] 
y_train = y_train[:-2000]

Creating an ImageDataGenerator instance for augmentation

In [31]:
datagen = ImageDataGenerator(
    rotation_range=10,            
    width_shift_range=0.1,        
    height_shift_range=0.1,       
    zoom_range=0.1,               
    shear_range=0.1,              
    horizontal_flip=False,         
    fill_mode='nearest'           
)
datagen.fit(x_train)

Compiling the Lenet model and using tanh activation function

In [32]:
model = models.Sequential()
model.add(layers.Conv2D(6, 5, activation='tanh', input_shape=x_train.shape[1:]))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(16, 5, activation='tanh'))
model.add(layers.AveragePooling2D(2))
model.add(layers.Activation('sigmoid'))
model.add(layers.Conv2D(120, 5, activation='tanh'))
model.add(layers.Flatten())
model.add(layers.Dense(84, activation='tanh'))
model.add(layers.Dense(10, activation='softmax'))
model.summary()

c:\Users\bhuva\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_9 (Conv2D)               │ (None, 28, 28, 6)      │           156 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_6             │ (None, 14, 14, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_6 (Activation)       │ (None, 14, 14, 6)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 10, 10, 16)     │         2,416 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_7             │ (None, 5, 5, 16)       │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_7 (Activation)       │ (None, 5, 5, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_11 (Conv2D)              │ (None, 1, 1, 120)      │        48,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 120)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 10)             │           850 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 61,706 (241.04 KB)

 Trainable params: 61,706 (241.04 KB)

 Non-trainable params: 0 (0.00 B)

Optimizing the model using Adam Optimizer and training the data till 40 epochs for accuracy and fitting the training dataset into augmented dataset

In [33]:
model.compile(optimizer='adam', loss=losses.sparse_categorical_crossentropy, metrics=['accuracy'])
history = model.fit(datagen.flow(x_train, y_train, batch_size=64),epochs=40,validation_data=(x_val, y_val))

Epoch 1/40


c:\Users\bhuva\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


907/907 ━━━━━━━━━━━━━━━━━━━━ 15s 16ms/step - accuracy: 0.1074 - loss: 2.3241 - val_accuracy: 0.7090 - val_loss: 1.0012
Epoch 2/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 14s 15ms/step - accuracy: 0.5588 - loss: 1.2712 - val_accuracy: 0.8865 - val_loss: 0.4194
Epoch 3/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.7543 - loss: 0.7572 - val_accuracy: 0.9310 - val_loss: 0.2592
Epoch 4/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 19s 21ms/step - accuracy: 0.8400 - loss: 0.5043 - val_accuracy: 0.9605 - val_loss: 0.1462
Epoch 5/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 29s 32ms/step - accuracy: 0.8798 - loss: 0.3800 - val_accuracy: 0.9630 - val_loss: 0.1299
Epoch 6/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 28s 31ms/step - accuracy: 0.8974 - loss: 0.3239 - val_accuracy: 0.9690 - val_loss: 0.1154
Epoch 7/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 30s 33ms/step - accuracy: 0.9048 - loss: 0.2970 - val_accuracy: 0.9675 - val_loss: 0.1104
Epoch 8/40
907/907 ━━━━━━━━━━━━━━━━━━━━ 23s 25ms/step - accuracy: 0.9175 - loss: 0.2582 - val_accurac

Now testing the model with the test dataset

In [34]:
model.evaluate(x_test, y_test)
print(f"Accuracy on test set: {model.evaluate(x_test, y_test)[1]:.2f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9776 - loss: 0.0675
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9776 - loss: 0.0675
Accuracy on test set: 0.98


Predicting labels on test set

In [35]:
y_pred_probs = model.predict(x_test)  
y_pred_classes = np.argmax(y_pred_probs, axis=1)  


313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step


Computing f1 Score
The f1 Score ranges from 1 to 0

In [36]:
f1 = f1_score(y_test, y_pred_classes, average='macro')
print(f"F1 Score (Weighted): {f1:.2f}")


F1 Score (Weighted): 0.98


After Image Augmentation:

    Accuracy : 0.98

    f1 Score : 0.98

After modifying the Training data we got the above accuracy and f1 score as the result